In [1]:
import numpy as np
import h5py 
import matplotlib.pyplot as plt
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D

In [31]:
candidates = pd.read_csv('candidatos_fase3_034.csv')
candidates
data = pd.read_csv('Simulation_034.csv')

In [32]:
for i in range(len(candidates)):
    data_reference_frame_changed = data.copy()
    data_reference_frame_changed['X'] -= candidates['X'][i]
    data_reference_frame_changed['Y'] -= candidates['Y'][i]
    data_reference_frame_changed['Z'] -= candidates['Z'][i]
    #Todas las galaxias que están dentro de la caja de lado 12R200
    caja = data_reference_frame_changed.loc[(abs(data_reference_frame_changed['X']) < 6* candidates['R200c'][i]) & (abs(data_reference_frame_changed['Y']) < 6* candidates['R200c'][i]) & (abs(data_reference_frame_changed['Z']) < 6* candidates['R200c'][i])]
    
    aislamiento = caja.loc[caja['MainHaloID'] != candidates['MainHaloID'][i]]
    print(aislamiento)
    
    # Modificar el DataFrame original según la condición
    #if len(aislamiento) != 0:
    #    candidates = candidates.drop(index=i)

      Unnamed: 0   MainHaloID       HaloID  Galaxy_Type          r        X  \
2418        2418  12568880793  12568880793            0 -15.863944   99.562   
2419        2419  12568880794  12568880794            0 -19.437637  353.182   
2423        2423  12568880798  12568880798            0 -16.662998  160.392   
2424        2424  12568880807  12568880807            0 -18.199590 -162.184   
2429        2429  12568880809  12568880809            0 -18.023138  107.937   
2437        2437  12568880842  12568880842            0 -17.930088  294.852   
2477        2477  12568880844  12568880844            2 -15.053790  235.112   
2482        2482  12568880844  12568880844            2 -14.472773  112.812   
2485        2485  12568880844  12568880844            2 -16.053970 -185.942   
2532        2532  12568880844  12568880843            2 -13.224082  244.382   
2534        2534  12568880844  12568880843            2 -14.238991  347.712   
2540        2540  12568880844  12568880843          

In [15]:
candidates.to_csv('Grupos_002.csv')

In [5]:
import h5py
import glob

# Ruta al directorio que contiene los archivos HDF5
directorio_archivos = '/home/seba/SAG'

# Patrón para buscar archivos HDF5
patron_archivos = '*.hdf5'  # O '*.hdf5' según la extensión de tus archivos

# Lista de archivos HDF5
lista_de_archivos = glob.glob(f'{directorio_archivos}/{patron_archivos}')

# DataFrame que contiene los grupos compactos
compact_groups = pd.read_csv('compact_groups.csv')

# Definir un conjunto para realizar una búsqueda más eficiente
mainhaloid_existente = set(compact_groups['MainHaloID'])

# Iterar sobre archivos HDF5
for archivo_nombre in lista_de_archivos:
    with h5py.File(archivo_nombre, "r") as archivo:
        # Transformar datos del archivo en un DataFrame
        data = pd.DataFrame({
            'MainHaloID': archivo['MainHaloID'][:].ravel(),
            'HaloID': archivo['HaloID'][:].ravel(),
            'R200': archivo['Halo/R200c'][:].ravel() * 0.674,
            'Galaxy_Type': archivo['Galaxy_Type'][:].ravel(),
            'r': archivo['Magnitudes/Mag_rS'][:].ravel(),
            'X': archivo['X'][:].ravel()*0.674,
            'Y': archivo['Y'][:].ravel()*0.674,
            'Z': archivo['Z'][:].ravel()*0.674,
            'Vx': archivo['Vx'][:].ravel(),
            'Vy': archivo['Vy'][:].ravel(),
            'Vz': archivo['Vz'][:].ravel()
            # Agrega otras columnas según sea necesario
        })

        # Filtrar las filas que coinciden con MainHaloID en compact_groups
        matching_rows = data[data['MainHaloID'].isin(mainhaloid_existente)]

        # Agregar las filas al DataFrame compact_groups solo si no existen
        compact_groups = pd.concat([compact_groups, matching_rows])

# Guardar el DataFrame compact_groups actualizado en un archivo CSV
compact_groups.to_csv('compact_groups_actualizado.csv', index=False)


In [6]:
g = pd.read_csv('compact_groups_actualizado.csv')
g

,Unnamed: 0,MainHaloID,HaloID,Galaxy_Type,r,X,Y,Z,Vx,Vy,...,m_rx,m_ry,m_rz,m_mean,Dispersion,Tamaño Angular del grupo XY (arcsec),Tamaño Angular del grupo YZ (arcsec),Tamaño Angular del grupo XZ (arcsec),Brillo Superficial,R200
0,50839.0,12573287739,12573287739,0,-20.794720,41750.945000,294551.300000,603763.90000,723.370000,-359.640000,...,16.572681,18.341423,18.114796,17.676300,176.256031,8.262238,3.658894,4.061396,21.323008,NaN
1,101668.0,12578000516,12578000516,0,-18.502258,53158.805000,26992.777000,26600.93400,153.090000,-26.520000,...,15.374680,14.390793,15.368244,15.044572,59.084737,14.495142,22.803343,14.538151,21.869802,NaN
2,212190.0,12588457573,12588457573,0,-21.445255,121460.710000,54554.890000,633406.40000,189.220000,98.040000,...,14.176449,17.571182,17.602364,16.449999,169.390297,13.380240,2.802324,2.762370,20.695171,NaN
3,20480.0,12570666350,12570666350,0,-20.662207,38017.010000,147525.980000,290428.47000,-505.170000,140.330000,...,15.251944,16.902208,16.671434,16.275194,164.734971,13.450189,6.290360,6.995689,21.612246,NaN
4,111396.0,12578732666,12578732666,0,-19.373373,59847.125000,41255.465000,244922.67000,111.340000,-882.510000,...,14.933969,17.602148,17.634737,16.723618,109.842368,15.271065,4.469247,4.402676,20.995966,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1055,NaN,12569144912,12569144911,1,-17.796637,42078.753906,43050.832031,571558.18750,559.080017,308.320007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.352993
1056,NaN,12578405581,12578405581,0,-19.919439,71223.343750,6426.570312,512893.28125,685.979980,733.450012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.612090
1057,NaN,12578405581,12578405581,2,-17.623331,71227.312500,6409.871094,512897.68750,595.189697,826.578796,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.722153
1058,NaN,12578405581,12578405581,2,-17.498795,71223.492188,6448.530273,512899.81250,749.101562,802.334900,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.001198


In [8]:
g = pd.read_csv('compact_groups_actualizado.csv')
g
# Contar el número de galaxias por MainHaloID
count_por_mainhaloid = g['MainHaloID'].value_counts()
#print(count_por_mainhaloid)
# Filtrar los MainHaloID que tienen 4, 5, 6, 7, 8 o 9 galaxias
mainhaloid_con_cantidad_especifica = count_por_mainhaloid[
    count_por_mainhaloid.isin([4, 5, 6, 7, 8, 9])
]

# Obtener la cantidad de grupos con 4, 5, 6, 7, 8 o 9 galaxias por separado
cantidad_grupos_4 = len(mainhaloid_con_cantidad_especifica[mainhaloid_con_cantidad_especifica == 4])
cantidad_grupos_5 = len(mainhaloid_con_cantidad_especifica[mainhaloid_con_cantidad_especifica == 5])
cantidad_grupos_6 = len(mainhaloid_con_cantidad_especifica[mainhaloid_con_cantidad_especifica == 6])
cantidad_grupos_7 = len(mainhaloid_con_cantidad_especifica[mainhaloid_con_cantidad_especifica == 7])
cantidad_grupos_8 = len(mainhaloid_con_cantidad_especifica[mainhaloid_con_cantidad_especifica == 8])
cantidad_grupos_9 = len(mainhaloid_con_cantidad_especifica[mainhaloid_con_cantidad_especifica == 9])
cantidad_grupos_10 = len(mainhaloid_con_cantidad_especifica[mainhaloid_con_cantidad_especifica == 10])


# Mostrar los resultados
print("Cantidad de grupos con 3 galaxias:", cantidad_grupos_4)
print("Cantidad de grupos con 4 galaxias:", cantidad_grupos_5)
print("Cantidad de grupos con 5 galaxias:", cantidad_grupos_6)
print("Cantidad de grupos con 6 galaxias:", cantidad_grupos_7)
print("Cantidad de grupos con 7 galaxias:", cantidad_grupos_8)
print("Cantidad de grupos con 8 galaxias:", cantidad_grupos_9)
print("Cantidad de grupos con 9 galaxias:", cantidad_grupos_9)


# Filtrar los MainHaloID que tienen 4, 5, 6, 7, 8 o 9 galaxias
mainhaloid_con_cantidad_especifica = count_por_mainhaloid[
    count_por_mainhaloid.isin([4, 5, 6, 7, 8, 9])
]


Cantidad de grupos con 3 galaxias: 0
Cantidad de grupos con 4 galaxias: 196
Cantidad de grupos con 5 galaxias: 11
Cantidad de grupos con 6 galaxias: 2
Cantidad de grupos con 7 galaxias: 0
Cantidad de grupos con 8 galaxias: 0
Cantidad de grupos con 9 galaxias: 0


In [7]:
cg = pd.read_csv('compact_groups.csv')
cg['R200c']

0     170.693050
1      60.013355
2     123.720960
3     142.297270
4      77.086130
         ...    
93     65.544480
94     72.478060
95    116.481460
96     74.078220
97     60.917652
Name: R200c, Length: 98, dtype: float64

In [83]:
grupitos = np.where(g["MainHaloID"] == 12569296101)[0]
grupitos

array([ 72, 492, 493, 494, 495])

In [84]:
g.iloc[grupitos]

,Unnamed: 0,MainHaloID,HaloID,Galaxy_Type,r,X,Y,Z,Vx,Vy,...,m_rx,m_ry,m_rz,m_mean,Dispersion,Tamaño Angular del grupo XY (arcsec),Tamaño Angular del grupo YZ (arcsec),Tamaño Angular del grupo XZ (arcsec),Brillo Superficial,R200
72,6080.0,12569296101,12569296101,0,-19.094376,40697.023000,52409.650000,83939.875000,77.540000,8.160000,...,15.014999,15.882877,15.754692,15.550857,93.880902,15.701269,10.528337,11.168556,21.487825,NaN
492,NaN,12569296101,12569296101,0,-19.094376,40697.023438,52409.648438,83939.875000,77.540001,8.160000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.351616
493,NaN,12569296101,12569296101,2,-18.640057,40694.726562,52402.726562,83902.625000,60.523006,-33.317986,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.993510
494,NaN,12569296101,12569296101,2,-16.447590,40715.402344,52410.843750,84040.007812,93.373917,52.931507,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.476994
495,NaN,12569296101,12569296101,2,-17.585222,40694.035156,52399.605469,83948.781250,-5.760765,48.180538,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.907352


In [62]:
grupitos

,0,1
0,12568660807,Unnamed: 0.1 Unnamed: 0 MainHaloID ...
1,12568698070,Unnamed: 0.1 Unnamed: 0 MainHaloID ...
2,12568739279,Unnamed: 0.1 Unnamed: 0 MainHaloID ...
3,12568778303,Unnamed: 0.1 Unnamed: 0 MainHaloID ...
4,12568879531,Unnamed: 0.1 Unnamed: 0 MainHaloID ...
...,...,...
93,12596669112,Unnamed: 0.1 Unnamed: 0 MainHaloID ...
94,12597103132,Unnamed: 0.1 Unnamed: 0 MainHaloID ...
95,12597180548,Unnamed: 0.1 Unnamed: 0 MainHaloID ...
96,12597764830,Unnamed: 0.1 Unnamed: 0 MainHaloID ...
